# 이미지의 바운딩 박스를 gui로 쉽게 그리자 아하

## 바운딩할 꽃 종류를 정하자

In [1]:
import os
import cv2
import numpy as np


flower_label = {'rose' : 0,             # 장미
                'sunflower': 1,         # 해바라기
                'tulip' : 2,            # 튤립
                'carnation' : 3,        # 카네이션
                'gerbera' : 4,          # 거베라
                'lily' : 5,             # 백합
                'freesia' : 6,          # 프리지아
                'chrysanthemum' : 7,    # 국화
                'peony' : 8,            # 작약
                'alstroemeria' : 9,     # 알스트로메리아
                'ranunculus' : 10,      # 라넌큘러스
                'lisianthus' : 11,      # 리시안셔스
                'iris' : 12,            # 붓꽃 (아이리스)
                'hydrangea' : 13,       # 수국
                'pompom blooms' :14,    # 퐁퐁국화
                'stock' : 15,           # 스톡
                'spary_rose' : 16,      # 스프레이 장미
                'ranunculus_butterfly':17,  # 라넌큘러스 버터플라이
                
                
                }

flower = 'gerbera'

f_num = flower_label[flower]

path = f'./0_flower_data/{flower}'

In [3]:
# 마우스 이벤트 콜백 함수
def mouse_callback(event, x, y, flags, param):
    global point1, point2, cnt, img, img2, coordi
    if event == cv2.EVENT_LBUTTONDOWN:
        if cnt%2:
            point1 = (x, y)
            img2 = img.copy()
            cv2.line(img, (x, y), (x, img.shape[0]), (0, 255, 0), 10)
            cv2.line(img, (x, y), (img.shape[1], y), (0, 255, 0), 10)
            cv2.imshow(f'{f_name} {width} * {height}', img)
        else:
            px, py = point1
            cv2.line(img2, (px, py), (px, y), (0, 255, 0), 10)
            cv2.line(img2, (px, py), (x, py), (0, 255, 0), 10)
            cv2.line(img2, (x, y), (px, y), (0, 255, 0), 10)
            cv2.line(img2, (x, y), (x, py), (0, 255, 0), 10)
            cv2.imshow(f'{f_name} {width} * {height}', img2)
            img = img2.copy()
            point2 = (x, y)
            
            if point1 and point2:
                # 좌측상단 좌표와 우측하단 좌표로 변환
                x1, y1 = min(point1[0], point2[0]), min(point1[1], point2[1])
                x2, y2 = max(point1[0], point2[0]), max(point1[1], point2[1])
                
                x = round((x1+(x2-x1)/2)/img.shape[1], 4)
                y = round((y1+(y2-y1)/2)/img.shape[0], 4)
                w = round((x2-x1)/img.shape[1], 4)
                h = round((y2-y1)/img.shape[0], 4)
                
                coordi += f"{f_num} {x} {y} {w} {h}\n"
                # 좌표 저장
        cnt += 1

break_plag = False
flower_list = os.listdir(path)

if not os.path.isdir(f'./1_labels_data/{flower}'):
    os.mkdir(f'./1_labels_data/{flower}')

for f_name in flower_list:
    
    if break_plag:
        break
    
    # 이미지 파일 경로
    img_path = f'./0_flower_data/{flower}/{f_name}'

    # 이미지 로드 및 윈도우 생성
    img = cv2.imread(img_path)
    
    if type(img) != np.ndarray:
        os.remove(f'./0_flower_data/{flower}/{f_name}')
        continue
    
    height, width = img.shape[:2]
    
    if height > width:
        new_height = 640
        new_width = int(width * (new_height / height))
    else:
        new_width = 640
        new_height = int(height * (new_width / width))
    
    cv2.namedWindow(f'{f_name} {width} * {height}', cv2.WINDOW_NORMAL)
    cv2.resizeWindow(f'{f_name} {width} * {height}', (new_width, new_height))
    cv2.setMouseCallback(f'{f_name} {width} * {height}', mouse_callback)
    cv2.imshow(f'{f_name} {width} * {height}', img)

    # 마우스 이벤트를 통한 좌표 저장
    point1, point2, coordi, cnt = None, None, '', 1
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        # 좌표를 txt 파일로 저장하고 다음으로 넘어가기
        # 만약 좌표를 그리지 않았다면 저장하지 않고 그냥 넘어가기
        if key == ord("q"):
            if coordi == '':
                break
            
            with open(f"./1_labels_data/{flower}/{f_name[:-4]}.txt", "a") as f:
                f.write(coordi)
            break
        
        # 저장하지 않은 좌표들을 지우고, 다시 그리기
        elif key == ord('c'):
            cnt = 1
            img = cv2.imread(img_path)
            coordi = ''
            cv2.imshow(f'{f_name} {width} * {height}', img)
        
        # 볼 것도 없다. 데이터셋에서 그냥 삭제
        elif key == ord('f'):
            os.remove(f'./0_flower_data/{flower}/{f_name}')
            break
        
        # 여기서 끝내기!
        elif key == ord('p'):
            break_plag = True
            break

    # 윈도우 종료 및 리소스 해제q
    cv2.destroyAllWindows()

## 바운딩 박스 그리기가 끝난 애들을 2로 넘기자

In [7]:
import shutil

bounding_txt = {i[:-4]:True for i in os.listdir(f'./1_labels_data/{flower}')}
img_list = [i[:-4] for i in os.listdir(f'./0_flower_data/{flower}')]

if not os.path.isdir(f'./2_learning_img/{flower}'):
    os.mkdir(f'./2_learning_img/{flower}')

for i in img_list:
    if bounding_txt.get(i):
        shutil.copy(f'./0_flower_data/{flower}/{i}.jpg', f'./2_learning_img/{flower}/{i}.jpg')

## 2의 이미지와 1의 라벨들을 3으로 나눠 보내자

In [8]:
import random

bounding_txt = os.listdir(f'./1_labels_data/{flower}')
img_list = os.listdir(f'./2_learning_img/{flower}')

random_list = random.sample([i for i in range(len(img_list))], len(img_list)//10*2)

for i in range(len(img_list)):
    CLS = 'train'
    label = bounding_txt[i]
    img = img_list[i]
    
    if i in random_list:
        CLS = 'test'
    
    shutil.move(f'./2_learning_img/{flower}/{img}', f'./3_one_time_data/{CLS}/images/{img}', )
    shutil.move(f'./1_labels_data/{flower}/{label}', f'./3_one_time_data/{CLS}/labels/{label}')

# 1차 학습 시작

# 이 아래는 파일 관리

### 3의 이전 파일들 삭제하기

In [4]:
for C in ['train', 'test']:
    for L in ['labels', 'images']:
        for i in os.listdir(f'./3_one_time_data/{C}/{L}'):
            os.remove(f'./3_one_time_data/{C}/{L}/{i}')
    
    if os.path.isfile(f'./3_one_time_data/{C}/labels.cache'):
        os.remove(f'./3_one_time_data/{C}/labels.cache')

### 3의 파일을 돌리는 코드

In [113]:
for C in ['train', 'test']:
    for L in ['labels', 'images']:
        for i in os.listdir(f'./3_one_time_data/{C}/{L}'):
            if L == 'images':
                shutil.move(f'./3_one_time_data/{C}/{L}/{i}', f'./2_learning_img/{flower}/{i}')
            else:
                shutil.move(f'./3_one_time_data/{C}/{L}/{i}', f'./1_labels_data/{flower}/{i}')
    
    if os.path.isfile(f'./3_one_time_data/{C}/labels.cache'):
        os.remove(f'./3_one_time_data/{C}/labels.cache')